In [136]:
QUESTIONS = [
    "How does CoRAG fundamentally differ from traditional RAG architectures in handling multi-hop reasoning, and what specific retrieval limitations does it address?",
    "Could the reliance on automated rejection sampling for intermediate chain generation introduce confirmation bias, and how does this compare to human-annotated reasoning chains?",
    "Given the 10+ point EM score improvements on HotpotQA/2WikiMultihopQA, what explains the inconsistent performance on Bamboogle where Search-o1-32B outperforms CoRAG?",
    "Why does CoRAG underperform on FEVER compared to Atlas-11B despite superior results elsewhere? Does this suggest task-specific limitations in fact verification?",
    "The paper claims a log-linear relationship between token consumption and performance - what safeguards prevent exponential compute costs in real-world deployments?",
    "If CoRAG's gains persist across retriever qualities (E5-base vs E5-large), does this imply the framework compensates for weak retrievers rather than synergizing with strong ones?",
    "Why does iterative rejection sampling improve 2WikiMultihopQA but degrade other datasets? Does this indicate diminishing returns in chain quality?",
    "The early stopping mechanism reduces tokens by 38% but lowers EM scores - how would you dynamically optimize this trade-off for production systems?",
    "Given the increased compute requirements (128k tokens for best performance), what environmental impact assessments were conducted for large-scale deployment?",
    "Could CoRAG's success on knowledge tasks transfer to creative domains, or does its reliance on retrieval inherently limit generative versatility compared to pure LLMs?"
]


### GraphRAG Search

In [11]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

In [12]:
INPUT_DIR = "../ragtest/output/"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [135]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="default-entity-description",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)

print(f"Entity count: {len(entity_df)}")

Entity count: 1653


In [134]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")

Relationship count: 919


In [133]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")

Report records: 131


In [132]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")

Text unit records: 53


In [17]:
# api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = "gpt-4o-mini" # os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = 'text-embedding-3-small' # os.environ["GRAPHRAG_EMBEDDING_MODEL"]


llm = ChatOpenAI(
    # api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    # api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

In [18]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None
    covariates=None,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [19]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [20]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [21]:
result = await search_engine.asearch(QUESTIONS[9])

In [ ]:
chunks_document_mapping = {}
chunks_document_mapping_r = {}
entity_document_mapping = {}
entity_document_mapping_r = {}
relationship_document_mapping = {}
relationship_document_mapping_r = {}

entities_df = pd.read_parquet('../ragtest/output/create_final_entities.parquet')
entity_mapper = {}
for idx, row in entity_df.iterrows():
    entity_mapper[row['id']] = row['human_readable_id']

relationship_mapper = {}
for idx, row in relationship_df.iterrows():
    relationship_mapper[row['id']] = row['human_readable_id']


for idx, row in text_unit_df.iterrows():
    chunks_document_mapping[row['human_readable_id']] = row['document_ids'][0]
    for e in row['entity_ids']:
        entity_document_mapping[entity_mapper[e]] = row['document_ids'][0]
    for r in row['relationship_ids']:
        relationship_document_mapping[relationship_mapper[r]] = row['document_ids'][0]


print(chunks_document_mapping[1])
print(entity_document_mapping[641])
print(relationship_document_mapping[638])

a8b765d9d64c173b527b19399d8071b2476bc8aa345827e0da4c7b7ddf7996876fa52fecaefaede23d41d6a43cbcde7efce40b55791eea8db599e6dd21253d73
f7ed019e71d299d7b11947cc8c588531072e3b2514bc473c4e1e6238ba392412c3a36c967222f35ce63231c7e21eebe88f6f2707129c2ff4f311575dd18046a3
f7ed019e71d299d7b11947cc8c588531072e3b2514bc473c4e1e6238ba392412c3a36c967222f35ce63231c7e21eebe88f6f2707129c2ff4f311575dd18046a3


In [111]:
def create_document_access_mapping():
    """Create a mock document access mapping with fictional user and document IDs."""
    return {
        "a8b765d9d64c173b527b19399d8071b2476bc8aa345827e0da4c7b7ddf7996876fa52fecaefaede23d41d6a43cbcde7efce40b55791eea8db599e6dd21253d73": ["user1", "admin"],
        "f7ed019e71d299d7b11947cc8c588531072e3b2514bc473c4e1e6238ba392412c3a36c967222f35ce63231c7e21eebe88f6f2707129c2ff4f311575dd18046a3": ["admin"],
    }

def filter_response_by_access(response_dict, access_level, user_id=None):
    """
    Filter the response dictionary based on access level and user permissions.
    
    Args:
        response_dict (dict): Original response containing all information
        access_level (str): One of 'KG_ONLY', 'CHUNKS', 'FULL', 'DOCUMENT_LEVEL'
        user_id (str): Required for DOCUMENT_LEVEL access checking
    
    Returns:
        dict: Filtered response based on access level
    """
    if access_level == "KG_ONLY":
        return {
            "relationships": response_dict["relationships"],
            "entities": response_dict["entities"]
        }
    
    elif access_level == "CHUNKS":
        return {
            "relationships": response_dict["relationships"],
            "entities": response_dict["entities"],
            "sources": response_dict["sources"]
        }
    
    elif access_level == "FULL":
        return response_dict
    
    elif access_level == "DOCUMENT_LEVEL":
        if not user_id:
            raise ValueError("user_id is required for DOCUMENT_LEVEL access")
        
        doc_access = create_document_access_mapping()
        accessible_docs = {doc_id for doc_id, users in doc_access.items() 
                         if user_id in users}
        print(accessible_docs)
        
        filtered_response = {}
        
        # Filter entities
        if "entities" in response_dict:
            filtered_response["entities"] = [
                entity for _, entity in response_dict["entities"].iterrows()
                if entity_document_mapping.get(int(entity["id"])) in accessible_docs
            ]
        
        # Filter relationships
        if "relationships" in response_dict:
            filtered_response["relationships"] = [
                rel for _, rel in response_dict["relationships"].iterrows()
                if relationship_document_mapping.get(int(rel["id"])) in accessible_docs
            ]
        
        # Filter sources
        if "sources" in response_dict:
            filtered_response["sources"] = [
                source for _, source in response_dict["sources"].iterrows()
                if chunks_document_mapping.get(int(source["id"])) in accessible_docs
            ]
        
        # Include reports if present and user has access
        if "reports" in response_dict:
            filtered_response["reports"] = response_dict["reports"]
        
        return filtered_response
    
    else:
        raise ValueError("Invalid access level")

In [113]:
# Example calls:
kg_only_result = filter_response_by_access(result.context_data, "KG_ONLY")
chunks_result = filter_response_by_access(result.context_data, "CHUNKS")
full_result = filter_response_by_access(result.context_data, "FULL")

In [139]:
for u in ["user1", "admin"]:
    document_level_result = filter_response_by_access(result.context_data, "DOCUMENT_LEVEL", user_id=u)
    print(f"User {u}:")
    print(pd.DataFrame(document_level_result['entities']).info())
    print(pd.DataFrame(document_level_result['relationships']).info())
    print(pd.DataFrame(document_level_result['sources']).info())
    print()
    print('#'*10)

{'a8b765d9d64c173b527b19399d8071b2476bc8aa345827e0da4c7b7ddf7996876fa52fecaefaede23d41d6a43cbcde7efce40b55791eea8db599e6dd21253d73'}
User user1:
<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 4 to 18
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       6 non-null      object
 1   entity                   6 non-null      object
 2   description              6 non-null      object
 3   number of relationships  6 non-null      object
 4   in_context               6 non-null      bool  
dtypes: bool(1), object(4)
memory usage: 246.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 3 to 56
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           20 non-null     object
 1   source       20 non-null     object
 2   target       20 non-null     object
 3   description  20 non-nu

### Create Output

In [140]:
from pathlib import Path
import ollama

def generate_search_prompt(context_data: str, response_type: str = "multiple paragraphs") -> str:
    """
    Generate a prompt using the local search system prompt template
    
    Args:
        context_data: The context data to include in the prompt
        response_type: The desired response format/length
    
    Returns:
        str: The formatted prompt
    """
    # Read template file
    template_path = Path("../ragtest/prompts/local_search_system_prompt.txt")
    with open(template_path, "r") as f:
        template = f.read()
    
    # Replace variables
    prompt = template.replace("{response_type}", response_type)
    prompt = prompt.replace("{context_data}", context_data)
    
    return prompt

In [143]:
def create_llm_response(model, prompt, question):
    # Use Ollama to test the prompt
    response = ollama.chat(
        model=model,  # or your preferred model
        messages=[{
            'role': 'user',
            'content': prompt
        }, {
            'role': 'user',
            'content': question
        }]
    )
    return response['message']['content']

In [147]:
for m in ['deepseek-r1:latest', 'phi3.5:latest']:
    for access in ["KG_ONLY", "CHUNKS", "FULL", "DOCUMENT_LEVEL"]:
        for u in ["user1", "admin"]:
            document_level_result = filter_response_by_access(result.context_data, access, user_id=u)
            # Generate prompt
            prompt = generate_search_prompt(
                context_data=str(document_level_result),
                response_type="multiple paragraphs"
            )
            response = create_llm_response(m, prompt, QUESTIONS[9])
            print(f"Model: {m} - Access: {access} - User: {u}")
            for k in document_level_result.keys():
                print(f"{k}: {len(pd.DataFrame(document_level_result[k]))}")
            print()
            print(response)
            print('#'*10)

Model: deepseek-r1:latest - Access: KG_ONLY - User: user1
relationships: 73
entities: 20

<think>
Okay, so I need to figure out whether CoRAG's success on knowledge tasks can be transferred to creative domains. The user is asking if the techniques used in CoRAG for knowledge tasks are applicable elsewhere, maybe even in areas like creativity.

First, let me recall what CoRAG does. From the data provided, it seems that CoRAG stands for something, maybe an acronym? Looking at the 'entities' table, I see mentions of RAG (Retrieval-Augmented Generation), LLMs (Large Language Models), and other techniques like knowledge engines and factual knowledge. So CoRAG is likely a method that enhances generation by using retrieval techniques to augment text.

Now, in the 'relationships' table, there are records linking CoRAG, RAG, LongRAG, and others with various concepts like retrieval-augmented generation (RAG), multi-task learning, and knowledge bases. It seems CoRAG is a specific approach within 

In [146]:
document_level_result.keys()

dict_keys(['relationships', 'entities'])

# Appendix

In [ ]:
from typing import List
from pydantic import BaseModel, Field
import instructor
from instructor import OpenAISchema
import openai
import chromadb
import ollama

entities = [
    "PERSON",
    "ORGANIZATION",
    "GEO",
    "EVENT",
    "MODEL",
    "TECHNIQUE",
    "DATASET",
    "METRIC",
    "BENCHMARK",
    "DOMAIN",
    "FRAMEWORK",
    "ALGORITHM",
    "PARAMETER",
    "OPTIMIZER",
    "LOSS_FUNCTION",
    "EMBEDDING",
    "RETRIEVER",
    "TUNING_METHOD"
]

# Patch the OpenAI client
client = instructor.patch(openai.OpenAI())

class EntityMatch(OpenAISchema):
    """Represents a matched entity from the query"""
    query_text: str = Field(description="The exact text segment from the query that matches an entity")
    entity_type: str = Field(description=f"The type of entity ({entities})")
    confidence: float = Field(description="Confidence score between 0 and 1", ge=0, le=1)

# Define the query and system prompt
system_prompt = """You are an expert entity matching system. Extract mentions of predefined entities from user queries.

            PREDEFINED ENTITIES:
            {entities}

            RULES:
            1. Only match entities from the provided list - do not suggest new entities
            2. Match variations and synonyms of entity names if they clearly refer to the same concept
            3. Assign confidence scores based on how certain the match is:
            - 1.0: Exact match
            - 0.7-0.9: Clear reference but different wording
            - 0.5-0.7: Possible match with some ambiguity
            4. If no entities are found, return an empty list
""".format(
    entities=entities
)

In [ ]:
# Make the API call
res = client.chat.completions.create(
    model="gpt-4o-mini",  # Make sure to use a valid model name
    response_model=List[EntityMatch],
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Extract entities from this query: {QUESTIONS[0]}"}
    ],
    temperature=0
)

# Print results
for entity in res:
    print((entity.query_text.upper(), entity.entity_type, entity.confidence))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


('CORAG', 'MODEL', 0.9)
('RAG', 'MODEL', 0.9)
('MULTI-HOP REASONING', 'TECHNIQUE', 0.7)
('RETRIEVAL LIMITATIONS', 'PARAMETER', 0.5)


### Semantic Embedding

In [ ]:
chroma_client = chromadb.PersistentClient(path="../chroma_db")
COLLECTION_NAME = 'knowledge_graph'

try:
    collection = chroma_client.get_collection(name=COLLECTION_NAME)
    print(f'Loaded collection ({COLLECTION_NAME})')
except:
    print('No Collection Found')

query_embedding = ollama.embed(model='all-minilm:33m', input=QUESTIONS[0])
results = collection.query(
    query_embeddings=query_embedding.embeddings,
    n_results=15,
)

Loaded collection (knowledge_graph)


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


### ReRanking

In [ ]:
from rerankers import Reranker
ranker = Reranker('flashrank')

Loading default flashrank model for language en
Default Model: ms-marco-MiniLM-L-12-v2
Loading FlashRankRanker model ms-marco-MiniLM-L-12-v2 (this message can be suppressed by setting verbose=0)
Loading model FlashRank model ms-marco-MiniLM-L-12-v2...


In [ ]:
rr_results = ranker.rank(query=QUESTIONS[0], docs=results['documents'][0], doc_ids=[x for x in range(len(results['documents'][0]))])
for r in rr_results.results:
    print(f"Rank: {r.doc_id} - {r.text}")

Rank: 3 - CoRAG utilizes retrieval chains to enhance its multi-hop question answering capabilities
Rank: 4 - CoRAG improves upon RAG by dynamically reformulating queries for better retrieval
Rank: 2 - CoRAG is particularly effective in the domain of multi-hop question answering, where it retrieves and reasons over multiple information sources.
Rank: 8 - CoRAG-8B utilizes retrieval chains to enhance its performance in multi-hop question answering tasks
Rank: 11 - CoRAG is a specific implementation of the retrieval-augmented generation framework, improving upon traditional methods.
Rank: 9 - RAG and CoRAG are related entities in the field of retrieval-augmented generation. The CoRAG framework outlines essential components for generating retrieval chains and training models within the RAG paradigm. CoRAG serves as an enhanced version of RAG, specifically designed to improve the retrieval process by integrating a chain-of-retrieval mechanism. This advancement allows for more efficient and 

In [17]:
# --> Communities
result.context_data['reports']

,id,title,content
0,20,Large Language Models and Their Ecosystem,# Large Language Models and Their Ecosystem\n\...


In [18]:
# Search By id
result.context_data['relationships'].head(3)

,id,source,target,description,weight,links,in_context
0,633,CORAG,RETRIEVAL-AUGMENTED GENERATION,CoRAG is a specific implementation of the retr...,9.0,1,True
1,638,RAG,CORAG,RAG and CoRAG are related entities in the fiel...,9.0,1,True
2,636,RAG,LONGRAG,LongRAG posits that RAG performance can be imp...,6.0,1,True


In [19]:
# Entities
result.context_data['entities'].head(3)

,id,entity,description,number of relationships,in_context
0,641,CORAG,CoRAG is a sophisticated model designed for mu...,23,True
1,660,RAG,Retrieval-Augmented Generation (RAG) is a doma...,8,True
2,37,RETRIEVAL-AUGMENTED GENERATION,Retrieval-Augmented Generation (RAG) is a soph...,40,True


In [20]:
# --> Chunks + 1
result.context_data['sources'].head(3)

,id,text
0,43,next-token prediction objective within a mult...
1,41,Chain-of-Retrieval Augmented Generation\n\nLia...
2,51,for both rejection sampling and test-time dec...


In [59]:
relationship_df.head(2)

,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,d9742a40-1414-4e30-9d1b-2cfc070a972e,0,VENKATESH BALAVADHANI PARTHASARATHY,CEADAR,Venkatesh Balavadhani Parthasarathy is affilia...,8.0,6,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
1,4860f12a-0b4f-49d1-a968-11e066ac2721,1,AHTSHAM ZAFAR,CEADAR,"Ahtsham Zafar is affiliated with CeADAR, contr...",8.0,6,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...


In [1]:
from arangodb_queries import *

# Initialize the ArangoDB client
client = ArangoClient(hosts="http://localhost:8529")

# Connect to "_system" database as root user
sys_db = client.db("_system", username="root", password="root")

# Create a new database if it doesn't exist
if not sys_db.has_database("knowledge_graph"):
    sys_db.create_database("knowledge_graph")

# Connect to the knowledge_graph database
db = client.db("knowledge_graph", username="root", password="root")

# Get the community with human_readable_id "20"
communities = get_communities_by_human_readable_ids([20, 21, 22], db)
relationships = get_relationships_by_human_readable_ids([633, 638], db)
entities = get_entities_by_human_readable_ids([1,2,3], db)
chunks = get_chunks_by_human_readable_ids([1,2,3], db)


In [2]:
print(communities)
print(relationships)
print(entities)
print(chunks)

[{'human_readable_id': 20, 'id': '6c9c1211-2f9f-436f-9234-19f153a4f320'}, {'human_readable_id': 21, 'id': '32432a3c-8507-4581-83c1-3cec0b8c61bd'}, {'human_readable_id': 22, 'id': '06795090-5cef-408c-9c28-384add4a608c'}]
[{'human_readable_id': 633, 'id': '77faed0e-660a-4384-9470-cdb94378c437'}, {'human_readable_id': 638, 'id': '210ab9e3-1d6b-4424-b033-ada92ace9e40'}]
[{'human_readable_id': 1, 'id': 'b131aa10-5aae-4723-8915-61ae01d3bddf'}, {'human_readable_id': 2, 'id': 'b06d9c88-a62b-4117-97e3-4aa390f13fce'}, {'human_readable_id': 3, 'id': '00a23f02-7c93-41fc-a0b9-9302b4f4d417'}]
[{'human_readable_id': 1, 'id': '9ed73c3fa03f83895a135beea0b9167417e1259afd8d6a2995037542ed2b04b6c11d37acdfa4bd0c280f586919f89c51a6bab22295153fd5f74f0faaccffd271'}, {'human_readable_id': 2, 'id': 'adb58fb2ac4dc2e636420a23765cd5f718dd4f88e00555d7aa84533fc852b568bd9b80d652ce717022631ce0aeb9d9b625d6d8020e55b7eb40d912c43dc13616'}, {'human_readable_id': 3, 'id': '759dea32dd8bddec032bdfe47d91384a521198df10389687cc5b0